# Negative Weighted Events

In this notebook, we adapt the negative events-based measure

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.algo.discovery.ocpn import algorithm as ocpn_discovery_factory
from src.utils import get_happy_path_log, create_flower_model, generate_variant_model
from ocpa.objects.log.importer.csv import factory as ocel_import_factory_csv
from models.negative_events_measure_without_weighting import dfs, negative_events_without_weighting
import pickle

# P2P Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [122]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
#ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})
#happy_path__ocel = get_happy_path_log(filename)
#ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})
# ots = ["order","item","delivery"]
# ocpn = create_flower_model(filename,ots)
filename_variant = "../src/data/csv/order_process_variant_log.csv" 
object_types = ["order","item","delivery"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)
filename = "../src/data/jsonocel/order_process.jsonocel"
ots = ["order","item","delivery"]
ocel = ocel_import_factory.apply(filename)
ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/order_variants/order_variant',object_types = ots,save_path_visuals=f"../reports/figures/order_variant_total.svg" )

Processing Variant Nets: 100%|██████████| 12/12 [00:00<00:00, 6004.01it/s]

#########Start generating Object-Centric Petri Net#########
#########Finished generating Object-Centric Petri Net#########


In [123]:
#since the process execution mappings have lists of length one, 
#we create another dictionary that only contains the the value inside the list
mapping_dict = {key: ocel.process_execution_mappings[key][0] for key in ocel.process_execution_mappings}

In [124]:
#we generate a new column in the class that contains the process executions via the generated dictionary
ocel.log.log['event_execution'] = ocel.log.log.index.map(mapping_dict)

In [125]:
events = np.unique(ocel.log.log.event_activity)

In [126]:
transitions = ocpn.transitions
# dictionary to store each activity as key and a list of its prior states/places as value
targets = {}
# dictionary to store each activity as key and a list of its following states/places as value

sources = {}
for arc in tqdm(ocpn.arcs, desc="Check the arcs"):
    # for each arc, check if our target is a valid (non-silent) transition
    if arc.target in transitions:
        # load all the prior places of a valid transition into a dictionary, where the key is the transition and the value
        # a list of all directly prior places
        if arc.target.name in targets:
            targets[arc.target.name].append(arc.source.name)
        else:
            targets[arc.target.name] = [arc.source.name]
    if arc.source in transitions:
        # load all the following places of a valid transition into a dictionary, where the key is the transition and the value
        # a list of all directly following places
        if arc.source.name in sources:
            sources[arc.source.name].append(arc.target.name)
        else:
            sources[arc.source.name] = [arc.target.name]

Check the arcs: 100%|██████████| 378/378 [00:00<00:00, 20423.65it/s]


In [127]:
preceding_activities = {}

In [128]:
preceding_activities = {}
for target_key, target_value in targets.items():
    preceding_activities[target_key] = []
    for source_key, source_value in sources.items():
        for element in target_value:
            if element in source_value:
                preceding_activities[target_key].append(source_key)
                break

In [129]:
preceding_activities

{'Fuel Car_0': ['Load Cargo_0'],
 'Start Route_0': ['Load Cargo_0', 'Fuel Car_0'],
 'Confirm Order_0': ['Place Order_0'],
 'End Route_0': ['Start Route_0'],
 'Load Cargo_0': ['Pick Item_0'],
 'Pay Order_0': ['Confirm Order_0'],
 'Pick Item_0': ['Confirm Order_0'],
 'Place Order_0': [],
 'Place Order_1': [],
 'Reorder Item_1': ['Item out of stock_1'],
 'Fuel Car_1': ['Load Cargo_1'],
 'Pay Order_1': ['Confirm Order_1'],
 'Load Cargo_1': ['Pick Item_1'],
 'Item out of stock_1': ['Confirm Order_1'],
 'Start Route_1': ['Fuel Car_1', 'Load Cargo_1'],
 'End Route_1': ['Start Route_1'],
 'Confirm Order_1': ['Place Order_1'],
 'Pick Item_1': ['Reorder Item_1'],
 'Start Route_2': ['Load Cargo_2', 'Fuel Car_2'],
 'End Route_2': ['Start Route_2'],
 'Place Order_2': [],
 'Fuel Car_2': ['Load Cargo_2'],
 'Payment Reminder_2': ['Confirm Order_2'],
 'Pay Order_2': ['Payment Reminder_2'],
 'Load Cargo_2': ['Pick Item_2'],
 'Confirm Order_2': ['Place Order_2'],
 'Pick Item_2': ['Confirm Order_2'],
 'Pa

In [130]:
succeeding_activities = {}

In [131]:
succeeding_activities = {}
for source_key, source_value in sources.items():
    succeeding_activities[source_key] = []
    for target_key, target_value in targets.items():
        for element in source_value:
            if element in target_value:
                succeeding_activities[source_key].append(target_key)
                break

In [132]:
succeeding_activities

{'Place Order_0': ['Confirm Order_0'],
 'Start Route_0': ['End Route_0'],
 'Load Cargo_0': ['Fuel Car_0', 'Start Route_0'],
 'End Route_0': [],
 'Confirm Order_0': ['Pay Order_0', 'Pick Item_0'],
 'Fuel Car_0': ['Start Route_0'],
 'Pick Item_0': ['Load Cargo_0'],
 'Pay Order_0': [],
 'End Route_1': [],
 'Reorder Item_1': ['Pick Item_1'],
 'Start Route_1': ['End Route_1'],
 'Pick Item_1': ['Load Cargo_1'],
 'Fuel Car_1': ['Start Route_1'],
 'Load Cargo_1': ['Fuel Car_1', 'Start Route_1'],
 'Place Order_1': ['Confirm Order_1'],
 'Pay Order_1': [],
 'Confirm Order_1': ['Pay Order_1', 'Item out of stock_1'],
 'Item out of stock_1': ['Reorder Item_1'],
 'Place Order_2': ['Confirm Order_2'],
 'Confirm Order_2': ['Payment Reminder_2', 'Pick Item_2'],
 'Load Cargo_2': ['Start Route_2', 'Fuel Car_2'],
 'Pick Item_2': ['Load Cargo_2'],
 'Fuel Car_2': ['Start Route_2'],
 'End Route_2': [],
 'Pay Order_2': [],
 'Start Route_2': ['End Route_2'],
 'Payment Reminder_2': ['Pay Order_2'],
 'Confirm Ord

In [133]:
#silent transitions
silent_transitions = [x.name for x in ocpn.transitions if  x.silent]

In [134]:
#replace the silent transitions in the succeeding activities dictionary
# Create a new dictionary to store the modified values
succeeding_activities_updated = {}

# Iterate through the dictionary
for key, values in succeeding_activities.items():
    # Create a list to store the modified values for this key
    new_values = []
    # Iterate through the values of each key
    for i in range(len(values)):
        # Check if the value is in the list of silent transitions
        if values[i] in silent_transitions:
            # Replace the value with the corresponding value from the dictionary
            new_values.extend(succeeding_activities[values[i]])
        else:
            # If the value is not in the list of silent transitions, add it to the new list
            new_values.append(values[i])
    # Add the modified values to the new dictionary
    succeeding_activities_updated[key] = new_values

In [135]:
succeeding_activities_updated

{'Place Order_0': ['Confirm Order_0'],
 'Start Route_0': ['End Route_0'],
 'Load Cargo_0': ['Fuel Car_0', 'Start Route_0'],
 'End Route_0': [],
 'Confirm Order_0': ['Pay Order_0', 'Pick Item_0'],
 'Fuel Car_0': ['Start Route_0'],
 'Pick Item_0': ['Load Cargo_0'],
 'Pay Order_0': [],
 'End Route_1': [],
 'Reorder Item_1': ['Pick Item_1'],
 'Start Route_1': ['End Route_1'],
 'Pick Item_1': ['Load Cargo_1'],
 'Fuel Car_1': ['Start Route_1'],
 'Load Cargo_1': ['Fuel Car_1', 'Start Route_1'],
 'Place Order_1': ['Confirm Order_1'],
 'Pay Order_1': [],
 'Confirm Order_1': ['Pay Order_1', 'Item out of stock_1'],
 'Item out of stock_1': ['Reorder Item_1'],
 'Place Order_2': ['Confirm Order_2'],
 'Confirm Order_2': ['Payment Reminder_2', 'Pick Item_2'],
 'Load Cargo_2': ['Start Route_2', 'Fuel Car_2'],
 'Pick Item_2': ['Load Cargo_2'],
 'Fuel Car_2': ['Start Route_2'],
 'End Route_2': [],
 'Pay Order_2': [],
 'Start Route_2': ['End Route_2'],
 'Payment Reminder_2': ['Pay Order_2'],
 'Confirm Ord

In [136]:
#recursive implementation of a depth-first search (DFS) algorithm
def dfs(graph, visited, activity, preceding_events):
    #takes as input the activity graph (represented as a dictionary), a set of visited nodes, the current activity, and a list to store the preceding events.
    visited.add(activity)
    for preceding_event in graph[activity]:
        #eighboring activity has not been visited yet, the algorithm visits it by calling the dfs function with the neighboring activity as the current activity.
        if preceding_event not in visited:
            dfs(graph, visited, preceding_event, preceding_events)
    preceding_events.append(activity)

preceding_events_dict = {}

# use a depth-first search (DFS) algorithm to traverse the activity graph and 
#create a list of all preceding events for each activity in the dictionary for directly preceding activities 
for activity in preceding_activities:
    #empty set for all the visited activities
    visited = set()
    #list for all the preceding events
    preceding_events = []
    dfs(preceding_activities, visited, activity, preceding_events)
    #we need to remove the last element from the list because it corresponds to the activity itself
    preceding_events_dict[activity] = preceding_events[:-1][::-1]

In [137]:
preceding_events_dict

{'Fuel Car_0': ['Load Cargo_0',
  'Pick Item_0',
  'Confirm Order_0',
  'Place Order_0'],
 'Start Route_0': ['Fuel Car_0',
  'Load Cargo_0',
  'Pick Item_0',
  'Confirm Order_0',
  'Place Order_0'],
 'Confirm Order_0': ['Place Order_0'],
 'End Route_0': ['Start Route_0',
  'Fuel Car_0',
  'Load Cargo_0',
  'Pick Item_0',
  'Confirm Order_0',
  'Place Order_0'],
 'Load Cargo_0': ['Pick Item_0', 'Confirm Order_0', 'Place Order_0'],
 'Pay Order_0': ['Confirm Order_0', 'Place Order_0'],
 'Pick Item_0': ['Confirm Order_0', 'Place Order_0'],
 'Place Order_0': [],
 'Place Order_1': [],
 'Reorder Item_1': ['Item out of stock_1', 'Confirm Order_1', 'Place Order_1'],
 'Fuel Car_1': ['Load Cargo_1',
  'Pick Item_1',
  'Reorder Item_1',
  'Item out of stock_1',
  'Confirm Order_1',
  'Place Order_1'],
 'Pay Order_1': ['Confirm Order_1', 'Place Order_1'],
 'Load Cargo_1': ['Pick Item_1',
  'Reorder Item_1',
  'Item out of stock_1',
  'Confirm Order_1',
  'Place Order_1'],
 'Item out of stock_1': ['

In [64]:
predecing_activities_full = {}

In [138]:
#delete all possible silent transitions from preceding_events_dict (dict where all direct preceeding events are stored)
filtered_preceeding_events_full = {}

for key, values in preceding_events_dict.items():
    if key not in silent_transitions:
        new_values = [val for val in values if val not in silent_transitions]
        filtered_preceeding_events_full[key] = new_values

In [139]:
filtered_preceeding_events_full

{'Fuel Car_0': ['Load Cargo_0',
  'Pick Item_0',
  'Confirm Order_0',
  'Place Order_0'],
 'Start Route_0': ['Fuel Car_0',
  'Load Cargo_0',
  'Pick Item_0',
  'Confirm Order_0',
  'Place Order_0'],
 'Confirm Order_0': ['Place Order_0'],
 'End Route_0': ['Start Route_0',
  'Fuel Car_0',
  'Load Cargo_0',
  'Pick Item_0',
  'Confirm Order_0',
  'Place Order_0'],
 'Load Cargo_0': ['Pick Item_0', 'Confirm Order_0', 'Place Order_0'],
 'Pay Order_0': ['Confirm Order_0', 'Place Order_0'],
 'Pick Item_0': ['Confirm Order_0', 'Place Order_0'],
 'Place Order_0': [],
 'Place Order_1': [],
 'Reorder Item_1': ['Item out of stock_1', 'Confirm Order_1', 'Place Order_1'],
 'Fuel Car_1': ['Load Cargo_1',
  'Pick Item_1',
  'Reorder Item_1',
  'Item out of stock_1',
  'Confirm Order_1',
  'Place Order_1'],
 'Pay Order_1': ['Confirm Order_1', 'Place Order_1'],
 'Load Cargo_1': ['Pick Item_1',
  'Reorder Item_1',
  'Item out of stock_1',
  'Confirm Order_1',
  'Place Order_1'],
 'Item out of stock_1': ['

In [140]:
#delete all possible silent transitions from filtered_preceeding_events (dict where only direct preceeding events are stored)
filtered_preceeding_events = {}

for key, values in preceding_activities.items():
    if key not in silent_transitions:
        new_values = [val for val in values if val not in silent_transitions]
        filtered_preceeding_events[key] = new_values

In [141]:
filtered_preceeding_events

{'Fuel Car_0': ['Load Cargo_0'],
 'Start Route_0': ['Load Cargo_0', 'Fuel Car_0'],
 'Confirm Order_0': ['Place Order_0'],
 'End Route_0': ['Start Route_0'],
 'Load Cargo_0': ['Pick Item_0'],
 'Pay Order_0': ['Confirm Order_0'],
 'Pick Item_0': ['Confirm Order_0'],
 'Place Order_0': [],
 'Place Order_1': [],
 'Reorder Item_1': ['Item out of stock_1'],
 'Fuel Car_1': ['Load Cargo_1'],
 'Pay Order_1': ['Confirm Order_1'],
 'Load Cargo_1': ['Pick Item_1'],
 'Item out of stock_1': ['Confirm Order_1'],
 'Start Route_1': ['Fuel Car_1', 'Load Cargo_1'],
 'End Route_1': ['Start Route_1'],
 'Confirm Order_1': ['Place Order_1'],
 'Pick Item_1': ['Reorder Item_1'],
 'Start Route_2': ['Load Cargo_2', 'Fuel Car_2'],
 'End Route_2': ['Start Route_2'],
 'Place Order_2': [],
 'Fuel Car_2': ['Load Cargo_2'],
 'Payment Reminder_2': ['Confirm Order_2'],
 'Pay Order_2': ['Payment Reminder_2'],
 'Load Cargo_2': ['Pick Item_2'],
 'Confirm Order_2': ['Place Order_2'],
 'Pick Item_2': ['Confirm Order_2'],
 'Pa

In [70]:
filtered_preceeding_events

{'Place Order': [],
 'Item out of stock': ['Confirm Order'],
 'Pick Item': ['Reorder Item'],
 'Confirm Order': ['Place Order'],
 'Start Route': ['Load Cargo', 'Fuel Car'],
 'Payment Reminder': [],
 'End Route': ['Start Route'],
 'Load Cargo': ['Pick Item'],
 'Pay Order': [],
 'Reorder Item': ['Item out of stock'],
 'Fuel Car': []}

In [142]:
grouped_df = ocel.log.log.sort_values('event_timestamp').groupby('event_execution')
DG = 0 #Disallowed Generalization
AG = 0 #Allowed Generalization
# Iterate over each group
for group_name, group_df in grouped_df:
    # Iterate over each row in the group
    # list for all the activities that are enabled, starting from all activities that do not have any preceeding activity
    enabled = [key for key, value in filtered_preceeding_events_full.items() if not value]
    # list of already executed activities
    trace =[]
    for index, row in group_df.iterrows():
        # Get the activity name and position
        negative_activities = [x for x in events if x != row['event_activity']]
        #it may happen that an activity is not enabled in the model but nevertheless executed in the log
        if row['event_activity'] in enabled:
            #check which elements in the negative activity list are enabled outside of the current activity
            enabled.remove(row['event_activity'])
        #get all the negative events that can not be executed in the process model at the moment
        disallowed = [value for value in negative_activities if value not in enabled]
        #add activity that has been executed to trace
        trace.append(row['event_activity'])
        AG = AG + len(enabled)
        DG = DG + len(disallowed)
        #may happen that activities in the log are not in the process model
        if row['event_activity'] in succeeding_activities_updated:
            #get all possible new enabled activities
            possible_enabled = succeeding_activities_updated[row['event_activity']]
            #check if each activity has more than one directly preceeding state
            for i in range(len(possible_enabled)):
                if len(filtered_preceeding_events[possible_enabled[i]]) < 2:
                    enabled.append(possible_enabled[i])
                else:
                    if all(elem in trace for elem in filtered_preceeding_events[possible_enabled[i]]):
                        enabled.append(possible_enabled[i])
        #extend the list with all elements that do not have any preceeding activity and are therefore enabled anyways
        enabled.extend([key for key, value in filtered_preceeding_events_full.items() if not value])
        #delete all duplicates from the enabled list
        enabled = list(set(enabled))
generalization = AG / (AG+DG)

In [143]:
print(generalization)

0.5652173913043478


In [23]:
enabled = [key for key, value in filtered_preceeding_events.items() if not value]

In [24]:
enabled

['Place Order', 'Fuel Car']

In [28]:
preceding_activities

{'Place Order': [],
 'Item out of stock': ['Confirm Order'],
 'orderskip_1': ['Confirm Order'],
 'orderinit_loop_2': ['Confirm Order'],
 'deliverytauSplit_1': [],
 'orderskip_5': ['Payment Reminder'],
 'itemskip_1': ['Confirm Order'],
 'Pick Item': ['Reorder Item', 'itemskip_1'],
 'Confirm Order': ['Place Order'],
 'orderskip_4': ['Payment Reminder'],
 'Start Route': ['Load Cargo', 'Fuel Car'],
 'Payment Reminder': ['orderskip_4', 'orderinit_loop_2'],
 'End Route': ['Start Route'],
 'Load Cargo': ['deliverytauSplit_1', 'Pick Item'],
 'Pay Order': ['orderskip_5', 'orderskip_1'],
 'Reorder Item': ['Item out of stock'],
 'Fuel Car': ['deliverytauSplit_1']}

In [13]:
generalization = negative_events_without_weighting (ocel, ocpn)

Check the arcs: 100%|██████████| 46/46 [00:00<?, ?it/s]
Calculate Generalization for all process executions: 100%|██████████| 48/48 [00:00<00:00, 269.96it/s]


In [14]:
print(generalization)

0.5063


In [ ]:
# generalization for pre-implemented ocpn process model: 0.50627
# generalization for flower model: 0.6925
# generalization for happy path model : 0.307378
# generalization for variant model: 0.56522